# Large problem with 1000 assets and 100 factors

In [1]:
import numpy as np
import cvxpy as cvx

from cvx.risk.factor import TimeseriesFactorRiskModel
from util.random import random_factors, random_beta, random_noise, random_assets

In [2]:
T = 2000
factors = random_factors(T=T, N=100, const_factor=False)

In [3]:
beta = random_beta(assets=random_assets(1000), factors=factors)

In [4]:
ret = factors @ beta + 0.01 * random_noise(factors @ beta)

In [5]:
triangle = TimeseriesFactorRiskModel(factors=factors, returns=ret)

In [8]:
#w = triangle.weights
w = cvx.Variable(shape=1000)
objective = cvx.Minimize(triangle.estimate_risk(w))
constraints = [w >= 0, cvx.sum(w) == 1]
problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(triangle.estimate_risk(w).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

1.0582242923942002e-07
